___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4


#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange


1. Підключитися до API.

In [1]:
import requests
import datetime as d
import math
import collections
import pickle

URL = 'https://bank.gov.ua/NBU_Exchange/exchange_site'
CURRENCY = 'usd'
END_DATE = d.date.today()
PERIOD_IN_DAYS = 365

def date_format(date):
    return date.strftime('%Y%m%d')


params ={
    "start": date_format(END_DATE-d.timedelta(days=PERIOD_IN_DAYS)),
    "end": date_format(END_DATE), 
    "valcode": CURRENCY,
    "sort": 'exchangedate',
    "order": 'desc',
    "json": 'json',
    }

response = (requests.get(url=URL, params=params)).json()

2. Отримати курс долара за останній рік.

In [2]:
rates_list =[[d.datetime.strptime(day['exchangedate'],'%d.%m.%Y').strftime('%m-%Y'), day['rate']] for day in response]

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [3]:
rates_by_month = collections.defaultdict(list)

for month, rate in rates_list:
    rates_by_month[month].append(rate)
       
rates_analysis_by_mounth = []

for month, rates in rates_by_month.items():
    current_rates_analysis ={}
    # 1. Находим среднее арифметическое выборки.
    average_rate = sum(rates) / len(rates)
    # 2. От каждого значения выборки отнимаем среднее арифметическое. 3. Каждую полученную разницу возводим в квадрат.
    # 4. Суммируем полученные значения квадратов разниц. 5. Делим на размер выборки. 6. Находим квадратный корень.
    std_dev = math.sqrt(sum([(rate - average_rate)**2 for rate in rates]) / (len(rates)))
    current_rates_analysis['month'] = month
    current_rates_analysis['average_rate'] = round(average_rate, 4)
    current_rates_analysis['std_dev'] = round(std_dev, 4)
    rates_analysis_by_mounth.append(current_rates_analysis)

rates_analysis_by_mounth

[{'month': '03-2023', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '02-2023', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '01-2023', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '12-2022', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '11-2022', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '10-2022', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '09-2022', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '08-2022', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '07-2022', 'average_rate': 31.8501, 'std_dev': 3.4993},
 {'month': '06-2022', 'average_rate': 29.2549, 'std_dev': 0.0},
 {'month': '05-2022', 'average_rate': 29.2549, 'std_dev': 0.0},
 {'month': '04-2022', 'average_rate': 29.2549, 'std_dev': 0.0},
 {'month': '03-2022', 'average_rate': 29.2549, 'std_dev': 0.0}]

4. Дану інформацію записати у файл за допомогою pickle.

In [8]:
with open('./rates_analysis.pickle', 'wb') as f:
    pickle.dump(rates_analysis_by_mounth, f)

with open('./rates_analysis.pickle','rb') as f:
    rates_analysis = pickle.load(f)

print ('rates_analysis', rates_analysis)

rates_analysis [{'month': '03-2023', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '02-2023', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '01-2023', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '12-2022', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '11-2022', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '10-2022', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '09-2022', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '08-2022', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '07-2022', 'average_rate': 31.8501, 'std_dev': 3.4993}, {'month': '06-2022', 'average_rate': 29.2549, 'std_dev': 0.0}, {'month': '05-2022', 'average_rate': 29.2549, 'std_dev': 0.0}, {'month': '04-2022', 'average_rate': 29.2549, 'std_dev': 0.0}, {'month': '03-2022', 'average_rate': 29.2549, 'std_dev': 0.0}]


### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/products

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [5]:
import requests
from bs4 import BeautifulSoup
import sqlite3


URL_AGROSEM = 'https://smallpacking.agrosem.ua/products/'

request_headers = {
    'authority': 'www.google.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
    'sec-ch-ua-mobile': '?0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'x-client-data': 'CIu2yQEIorbJAQjBtskBCKmdygEIrMfKAQj4x8oBCKPNygEI3NXKAQin+csB',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'uk',
    'Cookie': 'PHPSESSID=hec01mv9ejs0fi7grn0tpq9ndr' #Should be replaced from active session's cookies
}


def get_soup(url=URL_AGROSEM, headers=request_headers, page='', article=''):
    if len(page) > 0:
        page = '?page=' + page
    response_agrosem = requests.get(url=url + page + article,headers=headers)
    return BeautifulSoup(response_agrosem.text, 'lxml')

    
def count_page_number(start_page):
    while True:
        page_soup = get_soup(page=str(start_page))
        if page_soup.find('li', class_='next').find('a').get('href') is None:
            break
        start_page += 1
    return start_page

def get_article(item_info):
    return int(item_info.get('data-id'))


def get_item_name(item_info):
    return item_info.find('div',class_='info').get_text()[:-6].strip()


def get_item_price_in_uah(item_info):
    return float(''.join(item_info.find('p',class_='price').get_text()[:-1].strip().split()))


def get_item_weight(name='', article=''):
    if ' кг' in name or ' г' in name or ' мл' in name:
        return float(item_name.split()[-2])
    else:
        article_soup = get_soup(article=article) 
        return float(article_soup.find('table').find_all('td')[2].get_text().strip().split()[0])

    
def get_item_unit(name='', article=''):
    if ' кг' in name or ' г' in name or ' мл' in name:
        return item_name.split()[-1]
    else:
        article_soup = get_soup(article=article) 
        return article_soup.find('table').find_all('td')[2].get_text().strip().split()[1]
    
def insert_product_in_table(product):
    sql_insert= f"""INSERT INTO products (Article, Name, Price, Weight, Unit)
    VALUES({product[0]}, '{product[1]}',{product[2]},{product[3]},'{product[4]}'); """
    cur.execute(sql_insert)

In [6]:
page_num = count_page_number(1)
page_num

10

In [62]:
item_list = []

for i in range(page_num + 1):
    agro_soup = get_soup(page=str(i))
    for item in agro_soup.find_all('div', class_='item'):
        item_article = get_article(item)
        item_name = get_item_name(item)
        item_price = get_item_price_in_uah(item)
        item_weight = get_item_weight(name=item_name,article=str(item_article))
        item_unit = get_item_unit(name=item_name,article=str(item_article))
        
        item_tuple = (item_article,item_name,item_price,item_weight,item_unit)
        item_list.append(item_tuple)
 
# Article, Item_name, Price, Weight, Unit

In [82]:
#Зберегти в базу данних
agroserm_db = sqlite3.connect('./agroserm_products.sqlite')
cur = agroserm_db.cursor()

In [83]:
sql_create_table = """
    CREATE TABLE IF NOT EXISTS products (
    Id integer PRIMARY KEY AUTOINCREMENT,
	Article integer NOT NULL,
    Name text NOT NULL,
   	Price real NOT NULL,
	Weight real NOT NULL,
	Unit text NOT NULL
    );"""
cur.execute(sql_create_table)

In [85]:
for product in item_list:
    insert_product_in_table(product)

In [87]:
agroserm_db.close()

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

In [7]:
import sqlite3

db = sqlite3.connect('./tysql.sqlite')
cur = db.cursor()

In [8]:
def execute_sql_query(query):
    cur.execute(query)
    return cur.fetchall()

1. Виведіть інформацію про дану базу.

In [80]:
sql_DB_info = """PRAGMA database_list"""
execute_sql_query(sql_DB_info)

[(0, 'main', '/data/notebook_files/tysql.sqlite')]

In [21]:
sql_DB_info2 = """SELECT * FROM sqlite_schema"""
execute_sql_query(sql_DB_info2)

[('table',
  'Customers',
  'Customers',
  2,
  'CREATE TABLE Customers\n(\n  cust_id      char(10)  NOT NULL ,\n  cust_name    char(50)  NOT NULL ,\n  cust_address char(50)  NULL ,\n  cust_city    char(50)  NULL ,\n  cust_state   char(5)   NULL ,\n  cust_zip     char(10)  NULL ,\n  cust_country char(50)  NULL ,\n  cust_contact char(50)  NULL ,\n  cust_email   char(255) NULL ,\n  PRIMARY KEY (cust_id)\n)'),
 ('index', 'sqlite_autoindex_Customers_1', 'Customers', 3, None),
 ('table',
  'OrderItems',
  'OrderItems',
  4,
  'CREATE TABLE OrderItems\n(\n  order_num  int          NOT NULL                      ,\n  order_item int          NOT NULL                      ,\n  prod_id    char(10)     NOT NULL                      ,\n  quantity   int          NOT NULL                      ,\n  item_price decimal(8,2) NOT NULL                      ,\n  PRIMARY KEY (order_num, order_item)                   ,\n  FOREIGN KEY (order_num) REFERENCES Orders (order_num) ,\n  FOREIGN KEY (prod_id) REFEREN

In [22]:
sql_DB_info3 = """SELECT sqlite_version()"""
execute_sql_query(sql_DB_info3)

[('3.37.2',)]

2. Виведіть перелік всіх таблиць.

In [9]:
sql_list_tables = """SELECT tbl_name FROM sqlite_master WHERE type = 'table'"""
execute_sql_query(sql_list_tables)

[('Customers',), ('OrderItems',), ('Orders',), ('Products',), ('Vendors',)]

In [20]:
sql_list_tables2 = """SELECT name FROM sqlite_schema
WHERE type='table';"""

tables_list = execute_sql_query(sql_list_tables2)
tables_list

[('Customers',), ('OrderItems',), ('Orders',), ('Products',), ('Vendors',)]

In [30]:
# Всі таблиці з назвами колонок

info_schema=[]

for table in tables_list:
    tables_with_column_names_dict = {}
    table_name = str(table)[2:-3]
    sql_column_names = f"""PRAGMA table_info('{table_name}')"""
    table_columns = execute_sql_query(sql_column_names)
    column_names_list = []
    for column in table_columns:
        column_names_list.append(column[1])
    tables_with_column_names_dict[table_name] = column_names_list
    info_schema.append(tables_with_column_names_dict)


In [24]:
# Customers table
info_schema[0]

{'Customers': ['cust_id',
  'cust_name',
  'cust_address',
  'cust_city',
  'cust_state',
  'cust_zip',
  'cust_country',
  'cust_contact',
  'cust_email']}

In [25]:
# OrderItems table
info_schema[1]

{'OrderItems': ['order_num',
  'order_item',
  'prod_id',
  'quantity',
  'item_price']}

In [26]:
# Orders table
info_schema[2]

{'Orders': ['order_num', 'order_date', 'cust_id']}

In [27]:
# Products table
info_schema[3]

{'Products': ['prod_id', 'vend_id', 'prod_name', 'prod_price', 'prod_desc']}

In [28]:
# Vendors table
info_schema[4]

{'Vendors': ['vend_id',
  'vend_name',
  'vend_address',
  'vend_city',
  'vend_state',
  'vend_zip',
  'vend_country']}

3. Список всіх cust_id з таблиці Customers table.

In [48]:
sql_all_cust_id_from_customers = """SELECT cust_id FROM Customers;"""
execute_sql_query(sql_all_cust_id_from_customers)

[('1000000001',),
 ('1000000002',),
 ('1000000003',),
 ('1000000004',),
 ('1000000005',)]

4. Всю таблицю Customers table

In [50]:
sql_customer_table = """SELECT * FROM Customers;"""
execute_sql_query(sql_customer_table)

[('1000000001',
  'Village Toys',
  '200 Maple Lane',
  'Detroit',
  'MI',
  '44444',
  'USA',
  'John Smith',
  'sales@villagetoys.com'),
 ('1000000002',
  'Kids Place',
  '333 South Lake Drive',
  'Columbus',
  'OH',
  '43333',
  'USA',
  'Michelle Green',
  None),
 ('1000000003',
  'Fun4All',
  '1 Sunny Place',
  'Muncie',
  'IN',
  '42222',
  'USA',
  'Jim Jones',
  'jjones@fun4all.com'),
 ('1000000004',
  'Fun4All',
  '829 Riverside Drive',
  'Phoenix',
  'AZ',
  '88888',
  'USA',
  'Denise L. Stephens',
  'dstephens@fun4all.com'),
 ('1000000005',
  'The Toy Store',
  '4545 53rd Street',
  'Chicago',
  'IL',
  '54545',
  'USA',
  'Kim Howard',
  None)]

5. Список клієнтів (cust_names) відсортованих від Z до A

In [9]:
sql_customer_names = """SELECT cust_name 
                        FROM Customers 
                        ORDER BY cust_name DESC;"""
execute_sql_query(sql_customer_names)

[('Village Toys',),
 ('The Toy Store',),
 ('Kids Place',),
 ('Fun4All',),
 ('Fun4All',)]

6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [6]:
sql_customer_names = """SELECT c.cust_id, o.order_num
                        FROM Customers c
                        JOIN Orders o ON c.cust_id = o.cust_id
                        ORDER BY c.cust_id, o.order_date;"""
execute_sql_query(sql_customer_names)

[('1000000001', 20009),
 ('1000000001', 20005),
 ('1000000003', 20006),
 ('1000000004', 20007),
 ('1000000005', 20008)]

7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [17]:
sql_items = """SELECT quantity, item_price 
               FROM OrderItems 
               ORDER BY 1 desc, 2 desc """
execute_sql_query(sql_items)

[(250, 2.49),
 (250, 2.49),
 (250, 2.49),
 (100, 10.99),
 (100, 5.49),
 (100, 2.99),
 (100, 2.99),
 (100, 2.99),
 (50, 11.49),
 (50, 4.49),
 (20, 5.99),
 (10, 11.99),
 (10, 8.99),
 (10, 3.49),
 (10, 3.49),
 (10, 3.49),
 (5, 11.99),
 (5, 4.99)]

8. Товар (з таблиці Products), ціна якого становить 9.49

In [24]:
#Всі товари з цшною 9.49

sql_prod_price_all = """SELECT  prod_name 
                        FROM Products 
                        WHERE prod_price = 9.49"""
execute_sql_query(sql_prod_price_all)

[('King doll',), ('Queen doll',)]

In [25]:
#Якщо потрібен один товар, то треба обмежити виведення 1-ю стрічкою

sql_prod_price1 = """SELECT prod_name 
                     FROM Products 
                     WHERE prod_price = 9.49 
                     LIMIT 1"""
execute_sql_query(sql_prod_price1)

[('King doll',)]

In [26]:
#Якщо потрібен інший товар, то треба обмежити виведення 1-ю строчкою та змінити сортування

sql_prod_price2 = """SELECT prod_name 
                     FROM Products 
                     WHERE prod_price = 9.49 
                     ORDER BY 1 desc LIMIT 1"""
execute_sql_query(sql_prod_price2)

[('Queen doll',)]

9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [30]:
sql_prod_price3 = """SELECT prod_name,prod_price  
                     FROM Products WHERE prod_price >= 3 and prod_price <= 6 
                     ORDER BY prod_price asc"""
execute_sql_query(sql_prod_price3)

[('Fish bean bag toy', 3.49),
 ('Bird bean bag toy', 3.49),
 ('Rabbit bean bag toy', 3.49),
 ('Raggedy Ann', 4.99),
 ('8 inch teddy bear', 5.99)]

10. Кількість товару, що було продано

In [34]:
sql_sale_items = """SELECT sum(quantity) FROM OrderItems"""
execute_sql_query(sql_sale_items)

[(1430,)]

11. Кількість найменувань товару, ціна якого більша за 4

In [35]:
sql_prod_price4 = """SELECT count(*) FROM Products WHERE prod_price > 4"""
execute_sql_query(sql_prod_price4)

[(6,)]

12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [39]:
sql_3_most_expencive = """SELECT *  
                          FROM Products 
                          ORDER BY prod_price desc 
                          LIMIT 3"""
execute_sql_query(sql_3_most_expencive)

[('BR03',
  'BRS01',
  '18 inch teddy bear',
  11.99,
  '18 inch teddy bear, comes with cap and jacket'),
 ('RYL01',
  'FNG01',
  'King doll',
  9.49,
  '12 inch king doll with royal garments and crown'),
 ('RYL02',
  'FNG01',
  'Queen doll',
  9.49,
  '12 inch queen doll with royal garments and crown')]

13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [69]:
sql_order_num_by_cust = """SELECT cust_name, (
                            SELECT COUNT(*)
                            FROM Orders
                            WHERE cust_id = Customers.cust_id
                            ) AS order_count
                            FROM Customers
                            ORDER BY order_count DESC"""
execute_sql_query(sql_order_num_by_cust)

[('Village Toys', 2),
 ('Fun4All', 1),
 ('Fun4All', 1),
 ('The Toy Store', 1),
 ('Kids Place', 0)]

14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [70]:
# При використанні JOIN загубится остання стрічка, тут доцільніше використовувати LEFT JOIN
sql_order_num_by_cust2 = """SELECT c.cust_name, count(o.order_num) 
                            FROM Customers c 
                            LEFT JOIN Orders o ON c.cust_id=o.cust_id
                            GROUP BY c.cust_id
                            ORDER BY 2 DESC"""
execute_sql_query(sql_order_num_by_cust2)

[('Village Toys', 2),
 ('Fun4All', 1),
 ('Fun4All', 1),
 ('The Toy Store', 1),
 ('Kids Place', 0)]

15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [59]:
sql_item_list = """SELECT prod_name, prod_price 
                   FROM Products 
                   WHERE prod_price < 5
                   UNION
                   SELECT prod_name, prod_price 
                   FROM Products 
                   WHERE prod_price >= 5"""
execute_sql_query(sql_item_list)

[('12 inch teddy bear', 8.99),
 ('18 inch teddy bear', 11.99),
 ('8 inch teddy bear', 5.99),
 ('Bird bean bag toy', 3.49),
 ('Fish bean bag toy', 3.49),
 ('King doll', 9.49),
 ('Queen doll', 9.49),
 ('Rabbit bean bag toy', 3.49),
 ('Raggedy Ann', 4.99)]

### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди

Краткая информация о базе данных "Компьютерная фирма":

Схема БД состоит из четырех таблиц:\
Product(maker, model, type)\
PC(code, model, speed, ram, hd, cd, price)\
Laptop(code, model, speed, ram, hd, price, screen)\
Printer(code, model, color, type, price)\
 - Таблица Product представляет производителя (maker), номер модели (model) и тип ('PC' - ПК, 'Laptop' - ПК-блокнот или 'Printer' - принтер). Предполагается, что номера моделей в таблице Product уникальны для всех производителей и типов продуктов.\
 - В таблице PC для каждого ПК, однозначно определяемого уникальным кодом – code, указаны модель – model (внешний ключ к таблице Product), скорость - speed (процессора в мегагерцах), объем памяти - ram (в мегабайтах), размер диска - hd (в гигабайтах), скорость считывающего устройства - cd (например, '4x') и цена - price (в долларах).\ 
 - Таблица Laptop аналогична таблице РС за исключением того, что вместо скорости CD содержит размер экрана -screen (в дюймах).\
 - В таблице Printer для каждой модели принтера указывается, является ли он цветным - color ('y', если цветной), тип принтера - type (лазерный – 'Laser', струйный – 'Jet' или матричный – 'Matrix') и цена - price.

Задание: 1 (Serge I: 2002-09-30)
Найдите номер модели, скорость и размер жесткого диска для всех ПК стоимостью менее 500 дол. Вывести: model, speed и hd

In [ ]:
Select model,speed, hd
from PC
where price < 500

Задание: 2 (Serge I: 2002-09-21)
Найдите производителей принтеров. Вывести: maker

In [ ]:
Select distinct(maker)
from Product
where type = 'printer'

Задание: 3 (Serge I: 2002-09-30)
Найдите номер модели, объем памяти и размеры экранов ПК-блокнотов, цена которых превышает 1000 дол.

In [ ]:
Select model, ram, screen
from Laptop
where price > 1000

Задание: 4 (Serge I: 2002-09-21)
Найдите все записи таблицы Printer для цветных принтеров.

In [ ]:
Select *
from Printer
where color = 'y'

Задание: 5 (Serge I: 2002-09-30)
Найдите номер модели, скорость и размер жесткого диска ПК, имеющих 12x или 24x CD и цену менее 600 дол.

In [ ]:
Select model, speed, hd
from PC
where cd in('12x','24x') and price < 600

Задание: 6 (Serge I: 2002-10-28)
Для каждого производителя, выпускающего ПК-блокноты c объёмом жесткого диска не менее 10 Гбайт, найти скорости таких ПК-блокнотов. Вывод: производитель, скорость.

In [ ]:
Select distinct p.maker, l.speed
from Product p
join Laptop l on l.model = p.model
where l.hd >= 10 and p.type = 'Laptop'
order by p.maker, l.speed

Задание: 7 (Serge I: 2002-11-02)
Найдите номера моделей и цены всех имеющихся в продаже продуктов (любого типа) производителя B (латинская буква).

In [ ]:
Select l.model, l.price
from Product p
join Laptop l on l.model = p.model
where p.maker = 'B'
union
Select l.model, l.price
from Product p
join PC l on l.model = p.model
where p.maker = 'B'
union
Select l.model, l.price
from Product p
join Printer l on l.model = p.model
where p.maker = 'B'

Задание: 8 (Serge I: 2003-02-03)
Найдите производителя, выпускающего ПК, но не ПК-блокноты.

In [ ]:
Select distinct maker
from Product 
where type = 'PC' and maker not in (Select maker from Product where type = 'Laptop')

Задание: 9 (Serge I: 2002-11-02)
Найдите производителей ПК с процессором не менее 450 Мгц. Вывести: Maker

In [ ]:
Select distinct p.maker
from Product p
join PC pc on pc.model= p.model
where pc.speed >= 450

Задание: 10 (Serge I: 2002-09-23)
Найдите модели принтеров, имеющих самую высокую цену. Вывести: model, price

In [ ]:
Select model, price
from Printer 
where price = (select max(price) from Printer)

Задание: 11 (Serge I: 2002-11-02)
Найдите среднюю скорость ПК.

In [ ]:
Select avg(speed)
from PC

Задание: 12 (Serge I: 2002-11-02)
Найдите среднюю скорость ПК-блокнотов, цена которых превышает 1000 дол.

In [ ]:
select avg(speed)
from Laptop
where price > 1000

Задание: 13 (Serge I: 2002-11-02)
Найдите среднюю скорость ПК, выпущенных производителем A.

In [ ]:
Select avg(c.speed)
from PC c
join Product p on c.model = p.model
where p.maker = 'A'

Краткая информация о базе данных "Корабли":

Рассматривается БД кораблей, участвовавших во второй мировой войне. Имеются следующие отношения:\
Classes (class, type, country, numGuns, bore, displacement)\
Ships (name, class, launched)\
Battles (name, date)\
Outcomes (ship, battle, result)\
Корабли в «классах» построены по одному и тому же проекту, и классу присваивается либо имя первого корабля, построенного по данному проекту, либо названию класса дается имя проекта, которое не совпадает ни с одним из кораблей в БД. Корабль, давший название классу, называется головным.\
Отношение Classes содержит имя класса, тип (bb для боевого (линейного) корабля или bc для боевого крейсера), страну, в которой построен корабль, число главных орудий, калибр орудий (диаметр ствола орудия в дюймах) и водоизмещение ( вес в тоннах).\
В отношении Ships записаны название корабля, имя его класса и год спуска на воду.\
В отношение Battles включены название и дата битвы, в которой участвовали корабли.\
В отношении Outcomes – результат участия данного корабля в битве (потоплен-sunk, поврежден - damaged или невредим - OK).\
Замечания.
 1) В отношение Outcomes могут входить корабли, отсутствующие в отношении Ships. 
 2) Потопленный корабль в последующих битвах участия не принимает.

Задание: 14 (Serge I: 2002-11-05)
Найдите класс, имя и страну для кораблей из таблицы Ships, имеющих не менее 10 орудий.

In [ ]:
Select s.class, s.name, c.country
from Ships s
join Classes c on s.class = c.class
where c. numGuns >= 10

Задание: 15 (Serge I: 2003-02-03)
Найдите размеры жестких дисков, совпадающих у двух и более PC. Вывести: HD

In [ ]:
Select hd
from PC
group by hd
having count(hd) > 1

Задание: 16 (Serge I: 2003-02-03)
Найдите пары моделей PC, имеющих одинаковые скорость и RAM. В результате каждая пара указывается только один раз, т.е. (i,j), но не (j,i), Порядок вывода: модель с большим номером, модель с меньшим номером, скорость и RAM.

In [ ]:
Select distinct p.model, c.model, p.speed, p.ram
from PC p, PC c
where p.speed = c.speed and p.ram = c.ram and p.model > c.model
order by p.model desc, c.model desc

Задание: 17 (Serge I: 2003-02-03)
Найдите модели ПК-блокнотов, скорость которых меньше скорости каждого из ПК.
Вывести: type, model, speed

In [ ]:
select distinct p.type, l.model, l.speed
from Laptop l
join Product p on p.model = l.model
where l.speed < ALL(select speed from PC)

Задание: 18 (Serge I: 2003-02-03)
Найдите производителей самых дешевых цветных принтеров. Вывести: maker, price

In [ ]:
select distinct p1.maker, p2.price
from Product p1
join Printer p2 on p1.model = p2.model
where p2.color = 'y' and p2.price in (select min(price) from Printer where color = 'y')

Задание: 19 (Serge I: 2003-02-13)
Для каждого производителя, имеющего модели в таблице Laptop, найдите средний размер экрана выпускаемых им ПК-блокнотов.
Вывести: maker, средний размер экрана.

In [ ]:
select p.maker, avg(l.screen)
from Product p
join Laptop l on p.model = l.model
where p.type = 'laptop'
group by p.maker

Задание: 20 (Serge I: 2003-02-13)
Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК.

In [ ]:
select maker, count(distinct model) as num_models
from Product
where type = 'PC'
group by maker
having count(distinct model) >= 3

Задание: 21 (Serge I: 2003-02-13)
Найдите максимальную цену ПК, выпускаемых каждым производителем, у которого есть модели в таблице PC.
Вывести: maker, максимальная цена.

In [ ]:
select p2.maker, max(p1.price)
from PC p1
join Product p2 on p1.model=p2.model
group by p2.maker

Задание: 22 (Serge I: 2003-02-13)
Для каждого значения скорости ПК, превышающего 600 МГц, определите среднюю цену ПК с такой же скоростью. Вывести: speed, средняя цена.

In [ ]:
select speed, avg(price)
from PC
where speed > 600
group by speed

Задание: 23 (Serge I: 2003-02-14)
Найдите производителей, которые производили бы как ПК
со скоростью не менее 750 МГц, так и ПК-блокноты со скоростью не менее 750 МГц.
Вывести: Maker

In [ ]:
select distinct maker 
from Product 
where maker IN (
  select maker 
  from Product p
  join PC on p.model = PC.model 
  where PC.speed >= 750
  intersect
  select maker 
  from Product p
  join Laptop l on p.model = l.model 
  where l.speed >= 750
)
and type in ('PC', 'Laptop')

Задание: 24 (Serge I: 2003-02-03)
Перечислите номера моделей любых типов, имеющих самую высокую цену по всей имеющейся в базе данных продукции.

In [ ]:
with top_price as (select model, price
from PC 
where price =
(select max(price) price
from PC)
union
select model, price
from Laptop 
where price =
(select max(price) price
from Laptop)
union
select model, price
from Printer 
where price =
(select max(price) price
from Printer))

select model from top_price
where price = (select max(price) from top_price)

Задание: 25 (Serge I: 2003-02-14)
Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker

In [ ]:
select distinct maker
from Product
where type = 'Printer' and maker in( select maker
                                     from Product
                                     where model in(select model
                                                    from PC
                                                    where speed in (select max(speed) from PC) 
                                                    and ram in (select min(ram) from PC)
                                                    )
                                    )

Задание: 26 (Serge I: 2003-02-14)
Найдите среднюю цену ПК и ПК-блокнотов, выпущенных производителем A (латинская буква). Вывести: одна общая средняя цена.

In [ ]:
select avg(price) as AVG_price
from (
select price
from PC
where model in(
select model
from Product
where maker = 'A' and type = 'PC')
union all
select price
from Laptop
where model in(
select model
from Product
where maker = 'A' and type = 'Laptop')
) as prices

Задание: 27 (Serge I: 2003-02-03)
Найдите средний размер диска ПК каждого из тех производителей, которые выпускают и принтеры. Вывести: maker, средний размер HD.

In [ ]:
select maker, avg(hd) as AVG_hd
from Product p 
join PC on p.model = PC.model
where maker in (
select maker
from Product
where type = 'Printer')
group by maker

Задание: 28 (Serge I: 2012-05-04)
Используя таблицу Product, определить количество производителей, выпускающих по одной модели.

In [ ]:
select count(one_model_maker)
from (
select count(model) one_model_maker
from Product
group by maker
having count(maker) = 1) as one_model_maker_table

Задание: 29 (Serge I: 2003-02-14)
В предположении, что приход и расход денег на каждом пункте приема фиксируется не чаще одного раза в день [т.е. первичный ключ (пункт, дата)], написать запрос с выходными данными (пункт, дата, приход, расход). Использовать таблицы Income_o и Outcome_o.


Краткая информация о базе данных "Фирма вторсырья"

Фирма имеет несколько пунктов приема вторсырья. Каждый пункт получает деньги для их выдачи сдатчикам вторсырья. Сведения о получении денег на пунктах приема записываются в таблицу:\
Income_o(point, date, inc)\
Первичным ключом является (point, date). При этом в столбец date записывается только дата (без времени), т.е. прием денег (inc) на каждом пункте производится не чаще одного раза в день.\
Сведения о выдаче денег сдатчикам вторсырья записываются в таблицу:\
Outcome_o(point, date, out)
В этой таблице также первичный ключ (point, date) гарантирует отчетность каждого пункта о выданных деньгах (out) не чаще одного раза в день.\
В случае, когда приход и расход денег может фиксироваться несколько раз в день, используется другая схема с таблицами, имеющими первичный ключ code:\
Income(code, point, date, inc)\
Outcome(code, point, date, out)\
Здесь также значения столбца date не содержат времени.

In [ ]:
select coalesce(i.point, o.point) as point, coalesce(i.date, o.date) as date, i.inc, o.out
from Income_o i
full outer join Outcome_o o on i.point = o.point and i.date = o.date
order by point, date

Задание: 30 (Serge I: 2003-02-14)
В предположении, что приход и расход денег на каждом пункте приема фиксируется произвольное число раз (первичным ключом в таблицах является столбец code), требуется получить таблицу, в которой каждому пункту за каждую дату выполнения операций будет соответствовать одна строка.
Вывод: point, date, суммарный расход пункта за день (out), суммарный приход пункта за день (inc). Отсутствующие значения считать неопределенными (NULL).

In [ ]:
select coalesce(i.point, o.point) as point, coalesce(i.date, o.date) as date, o.outcome, i.income
from
  (select point, date, sum(inc) as income
   from Income
   group by point, date) as i
   full outer join
  (select point, date, sum(out) as outcome
   from Outcome
   group by point, date) as o
  on i.point = o.point and i.date = o.date
order by point, date